In [1]:
from mctnet.actions import locate_peaks
from mctnet.data_loading import _load_point_data
import torchio as tio
import numpy as np
from scipy import spatial
import napari

In [2]:
mct = tio.ScalarImage('./dataset/fiddler/cropped_with_csv_labs/test_images_whole_10/dampieri_male_16-image.nii.gz')
prediction = tio.ScalarImage('./output/dampieri_male_16-image.prediction_sigma_3.nii.gz')

In [3]:
prediction_locations = locate_peaks(
    './output/dampieri_male_16-image.prediction_sigma_3.nii.gz',
    save=True,
    plot=False,
    peak_min_dist=4,
    peak_min_val=0.5,
)

Locating peaks...
Saving peaks...


In [4]:
ground_truth_locations=np.loadtxt(
    './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
    delimiter=',',
    dtype=np.float
).astype(int).T

In [16]:
def get_acc_metrics(y_hat, y):
    """Calculates accuracy metrics for a set of predicted and ground truth coordinates.

    Is a true positive if the distance between the predicted and closest ground truth coordinate
    is less than the correct_prediction_distance config parameter. Is a false positive if the 
    distance is greater than the correct_prediction_distance parameter or it already has a closer
    true positive. Is a false negative if the ground truth does not have a corresponding true
    positive.

    Args:
        y_hat (np.ndarray): predicted coordinates
        y (np.ndarray): ground truth coordinates

    Returns:
        tp (float): true positives
        fp (float): false positives
        fn (float): false negatives
        loc_errs (np.ndarray): location errors
    """

    tree = spatial.cKDTree(y)
    closest_dists, closest_nbrs = tree.query(y_hat, k=1)

    # if predictions are within distance of the same point, only keep the first one
    # this is to avoid repeated counting of true positives that are actually false positives
    # it doesn't matter which one is closer in this case, as we are just making a count
    removed_dup_indx = np.unique(closest_nbrs, return_index=True)[1]
    mask = np.zeros(closest_nbrs.shape, dtype='bool')
    mask[removed_dup_indx] = True

    true_positive = (closest_dists <= 5) & mask

    tp = len(true_positive[true_positive])
    fp = len(true_positive[~true_positive])
    fn = y.shape[0] - tp
    loc_errors = closest_dists[true_positive]

    if len(loc_errors) == 0:
        loc_errors = np.array([0])

    tp_groundtruth = closest_nbrs[true_positive]
    fn_mask = np.ones(y.shape[0], dtype='bool')
    fn_mask[tp_groundtruth] = False

    all_ground_truth = y
    fp_prediction = y_hat[~true_positive]
    fn_to_plot = y[fn_mask]
    tp_groundtruth = y_hat[closest_nbrs[true_positive]]
    tp_prediction = y_hat[true_positive]

    things_to_plot = [all_ground_truth, fp_prediction, fn_to_plot, tp_groundtruth, tp_prediction]

    print(f'True positives: {tp}, False positives: {fp}, False negatives: {fn}, N Real values: {y.shape[0]}, N Predicted values: {y_hat.shape[0]}')
    print(f'Percent correctly predicted {tp / y.shape[0] * 100}%')
    print(f'Mean Localisation error: {loc_errors.mean()}')

    return tp, fp, fn, loc_errors, things_to_plot

tp, fp, fn, loc_errors, things_to_plot = get_acc_metrics(prediction_locations, ground_truth_locations)

viewer = napari.view_points(things_to_plot[0], name='all ground truth', size=2, face_color='pink')
viewer.add_points(things_to_plot[1], name='fp prediction', size=2, face_color='red')
viewer.add_points(things_to_plot[2], name='fn', size=2, face_color='yellow')
viewer.add_points(things_to_plot[3], name='tp groundtruth', size=2, face_color='blue')
viewer.add_points(things_to_plot[4], name='tp prediction', size=2, face_color='green')

True positives: 4545, False positives: 1298, False negatives: 712, N Real values: 5257, N Predicted values: 5843
Percent correctly predicted 86.4561536998288%
Mean Localisation error: 2.4422055624953827


/home/jake/projects/mctnet/venv/lib/python3.8/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Points layer 'tp prediction' at 0x7fa9103d3880>